# Weapon–Target Assignment via Piecewise Linearization and CPLEX

Este notebook muestra paso a paso cómo resolver el modelo WTA (Ecuación 3.1)

---

## 1. Modelo canónico

**Variables de decisión**  
- $x_{ij}\in\mathbb{Z}_+$: número de armas de tipo $i$ asignadas al blanco $j$.

**Parámetros**  
- $w_i$: inventario disponible de armas tipo $i$.  
- $V_j$: valor de destrucción del blanco $j$.  
- $p_{ij}$: probabilidad de impacto de un arma tipo $i$ sobre el blanco $j$.  
- Definimos $q_{ij} = 1 - p_{ij}$.

**Función objetivo**  
> Minimizar la suma de supervivencias ponderadas:  
>
> $$
> \min_{x\in\mathbb{Z}_+}\;F(x)
> \;=\;
> \sum_{j=1}^n V_j \,\prod_{i=1}^m (1 - p_{ij})^{\,x_{ij}}
> \;=\;
> \sum_{j=1}^n V_j \exp\!\Bigl(\sum_{i=1}^m x_{ij}\ln q_{ij}\Bigr).
> $$

**Restricciones**  
> Inventario de armas:
> $$
> \sum_{j=1}^n x_{ij} \;\le\; w_i
> \quad
> \forall\,i=1,\dots,m
> $$
>
> Dominio entero:
> $$
> x_{ij}\in\mathbb{Z}_+,
> \quad
> \forall\,i=1,\dots,m,\;j=1,\dots,n.
> $$

---

## 2. Linealización _Piecewise Linear_ (PWL)

Para cada blanco $j$, definimos la variable auxiliar  
$
y_j = \sum_{i=1}^m x_{ij}\,\ln q_{ij},
$
y aproximamos  
$
f_j = V_j\,e^{y_j}
$
mediante segmentos lineales:

1. **Bounds de** $y_j$  
   $
     \underline y_j = \sum_{i=1}^m w_i\,\ln(q_{ij}),
     \quad
     \overline y_j = 0.
   $

2. **Puntos de ruptura**  
   Para $k=0,1,\dots,K$:
   $
     \xi_{j,k}
     = \underline y_j + \frac{k}{K}\bigl(\overline y_j - \underline y_j\bigr),
     \quad
     f_{j,k} = V_j\,e^{\,\xi_{j,k}}.
   $

3. **Variables PWL**  
   $\lambda_{j,k}\ge0$, con
   $$
     \sum_{k=0}^K \lambda_{j,k} = 1,\quad
     y_j = \sum_{k=0}^K \xi_{j,k}\,\lambda_{j,k},\quad
     f_j = \sum_{k=0}^K f_{j,k}\,\lambda_{j,k}.
   $$

---

In [47]:
import math
import random
import time
import pandas as pd
import pulp

def solve_wta_pwl(
    w,        # lista de inventarios w[i] para cada tipo i
    n,        # número de blancos
    K,      # número de tramos PWL
    seed=218   # semilla para reproducibilidad
):
    random.seed(seed)
    m = len(w)

    # 1) Generar aleatoriamente p_{ij}∈[0.5,0.8] y V_j∈[5,10]
    p = [[random.uniform(0.5, 0.8) for j in range(n)] for i in range(m)]
    V = [random.uniform(5, 10)   for j in range(n)]

    # 2) Pre-cálculos
    q     = [[1 - p[i][j] for j in range(n)] for i in range(m)]
    y_min = [sum(w[i]*math.log(q[i][j]) for i in range(m)) for j in range(n)]
    breaks = {
        j: [y_min[j] + k*(0.0 - y_min[j])/K for k in range(K+1)]
        for j in range(n)
    }
    f_vals = {
        j: [V[j]*math.exp(y) for y in breaks[j]]
        for j in range(n)
    }

    # 3) Construir el modelo PWL en PuLP
    model = pulp.LpProblem("WTA_PWL", pulp.LpMinimize)

    # Variables
    x   = pulp.LpVariable.dicts("x",   (range(m), range(n)), lowBound=0, cat="Integer")
    y   = { j: pulp.LpVariable(f"y_{j}", lowBound=y_min[j], upBound=0.0)
            for j in range(n) }
    f_j = { j: pulp.LpVariable(f"f_{j}", lowBound=min(f_vals[j]), upBound=max(f_vals[j]))
            for j in range(n) }
    lam = { (j,k): pulp.LpVariable(f"lam_{j}_{k}", lowBound=0)
            for j in range(n) for k in range(K+1) }

    # 4) Restricciones PWL + enlace
    for j in range(n):
        model += pulp.lpSum(lam[(j,k)] for k in range(K+1)) == 1
        model += y[j]   == pulp.lpSum(breaks[j][k]  * lam[(j,k)] for k in range(K+1))
        model += f_j[j] == pulp.lpSum(f_vals[j][k] * lam[(j,k)] for k in range(K+1))
        model += y[j]   == pulp.lpSum(x[i][j] * math.log(q[i][j]) for i in range(m))

    # 5) Inventario de armas
    for i in range(m):
        model += pulp.lpSum(x[i][j] for j in range(n)) <= w[i]

    # 6) Objetivo
    model += pulp.lpSum(f_j[j] for j in range(n))

    # 7) Resolver con CPLEX (ajusta el path si es necesario)
    start = time.time()
    model.solve(pulp.CPLEX_CMD(
        path="/Applications/CPLEX_Studio_Community2211/cplex/bin/arm64_osx/cplex",
        msg=False
    ))
    status = pulp.LpStatus[model.status]
    print(f"Estado del solver: {status}")
    elapsed_exact = time.time() - start

    # 8) Leer solución
    if status != "Optimal":
        raise RuntimeError(f"CPLEX no encontró solución óptima (estado: {status})")
    sol = [[int(round(pulp.value(x[i][j]))) for j in range(n)] for i in range(m)]
    obj = pulp.value(model.objective)

    return sol, obj, p, V, elapsed_exact

# —————————————— USO ——————————————
if __name__ == "__main__":
    # Parámetros que puedes cambiar:
    w = [4, 3, 0, 3, 3, 4]   # inventario de armamento de 6 buques
    n = 12           # cantidad de amenazas (blancos)
    K = 70          # refinamos a 40 tramos PWL
    sol, obj, p, V, elapsed_exact = solve_wta_pwl(w, n, K, seed=218)

    print(f"Resultados Algoritmo Exacto - CPLEX (PWL)")
    print(f"Objetivo (suma supervivencia de las amenazas): {obj:.4f}")
    print(f"Tiempo de resolución (exacto): {elapsed_exact:.4f} segundos")
    print("Asignación x[i][j]:")
    for i in range(len(w)):
        for j in range(n):
            if sol[i][j]:
                print(f"  Buque {i} →amenaza {j}: {sol[i][j]} misil(es)")

    # Opcional: imprime p y V para referencia
    print("\np_ij (misil→blanco):")
    for row in p: 
        print(" ", ["{:.2f}".format(v) for v in row])
    print("\nV_j (valor de cada blanco):", ["{:.2f}".format(v) for v in V])


Estado del solver: Optimal
Resultados Algoritmo Exacto - CPLEX (PWL)
Objetivo (suma supervivencia de las amenazas): 14.8114
Tiempo de resolución (exacto): 0.0916 segundos
Asignación x[i][j]:
  Buque 0 →amenaza 2: 1 misil(es)
  Buque 0 →amenaza 5: 1 misil(es)
  Buque 0 →amenaza 7: 2 misil(es)
  Buque 1 →amenaza 0: 1 misil(es)
  Buque 1 →amenaza 9: 2 misil(es)
  Buque 3 →amenaza 1: 2 misil(es)
  Buque 3 →amenaza 6: 1 misil(es)
  Buque 4 →amenaza 4: 1 misil(es)
  Buque 4 →amenaza 6: 1 misil(es)
  Buque 4 →amenaza 10: 1 misil(es)
  Buque 5 →amenaza 3: 1 misil(es)
  Buque 5 →amenaza 8: 1 misil(es)
  Buque 5 →amenaza 11: 2 misil(es)

p_ij (misil→blanco):
  ['0.59', '0.53', '0.78', '0.60', '0.56', '0.77', '0.51', '0.72', '0.60', '0.56', '0.60', '0.57']
  ['0.69', '0.75', '0.64', '0.56', '0.57', '0.64', '0.54', '0.72', '0.54', '0.64', '0.68', '0.64']
  ['0.58', '0.77', '0.65', '0.76', '0.55', '0.76', '0.79', '0.58', '0.71', '0.74', '0.78', '0.68']
  ['0.67', '0.73', '0.63', '0.52', '0.55', '0.

## ⚙️ Greedy Heuristic for Weapon–Target Assignment (WTA)

### 🎯 Objective

We aim to solve the canonical WTA formulation:

$
\min \sum_{j=1}^{n} V_j \prod_{i=1}^{m} \left(1 - p_{ij} \right)^{x_{ij}}
$

Where:

- $ V_j $: Destruction value of target $ j $
- $ p_{ij} $: Probability of weapon $ i $ destroying target $ j $
- $ x_{ij} \in \mathbb{Z}_+ $: Number of weapons $ i $ assigned to target $ j $

This can be rewritten using survival probabilities $ q_{ij} = 1 - p_{ij} $:

$
\min \sum_{j=1}^{n} V_j \prod_{i=1}^{m} q_{ij}^{x_{ij}} = \sum_{j=1}^{n} V_j \cdot \exp\left( \sum_{i=1}^{m} x_{ij} \log q_{ij} \right)
$

Let:

$
y_j = \sum_{i=1}^{m} x_{ij} \log q_{ij}, \quad f_j = V_j \cdot e^{y_j}
$

Then the objective becomes:

$
\min \sum_{j=1}^{n} f_j
$

---

### 💡 Greedy Algorithm Strategy

We use a simple greedy heuristic to approximate the optimal solution:

1. **Initialization**:
   - Set all $ x_{ij} = 0 $
   - Initialize inventory $ w_i $ for each weapon type

2. **Iteratively assign one weapon**:
   - For each feasible assignment $ (i,j) $, compute the **marginal increase in survival**:
     $
     \Delta f_j = f_j^{\text{new}} - f_j^{\text{current}}
     $
   - Choose the assignment with the **minimum** $ \Delta f_j $
   - Update $ x_{ij} $ and decrement inventory of weapon $ i $

3. **Stop when no weapons remain** or no improvement is possible.

---

### 🧪 Parameters Used in the Code

- Number of weapon types: `m = len(w)`
- Inventory per weapon type: `w = [4, 3, 5]`
- Number of targets: `n = 7`
- Random seed: `218` (to ensure comparability with exact model)
- Probabilities $ p_{ij} \sim \mathcal{U}(0.5, 0.8) $
- Values $ V_j \sim \mathcal{U}(5, 10) $

---

### ✅ Output Format

The output includes:

- Total objective (sum of expected survivals)
- Assignment matrix $ x_{ij} $
- Probabilities $ p_{ij} $ used
- Target values $ V_j $

In [44]:
import math
import random

def solve_wta_greedy(w, n, seed=218):
    random.seed(seed)
    m = len(w)

    # Generar datos igual que el modelo exacto
    p = [[random.uniform(0.5, 0.8) for j in range(n)] for i in range(m)]
    V = [random.uniform(5, 10) for j in range(n)]

    # Calcular q_{ij}
    q = [[1 - p[i][j] for j in range(n)] for i in range(m)]

    # Inicialización
    x = [[0 for j in range(n)] for i in range(m)]
    remaining = w[:]

    # Estrategia greedy
    start = time.time()
    while True:
        best_delta = float("inf")
        best_i, best_j = None, None

        for i in range(m):
            if remaining[i] == 0:
                continue
            for j in range(n):
                current_yj = sum(x[k][j] * math.log(q[k][j]) for k in range(m))
                current_fj = V[j] * math.exp(current_yj)

                proposed_yj = current_yj + math.log(q[i][j])
                proposed_fj = V[j] * math.exp(proposed_yj)
                delta = proposed_fj - current_fj

                if delta < best_delta:
                    best_delta = delta
                    best_i, best_j = i, j

        if best_i is None:
            break

        x[best_i][best_j] += 1
        remaining[best_i] -= 1

    # Objetivo total
    total_f = 0
    for j in range(n):
        y_j = sum(x[i][j] * math.log(q[i][j]) for i in range(m))
        total_f += V[j] * math.exp(y_j)
    elapsed_greedy = time.time() - start

    # Mostrar resultados
    print(f"Resultados Heurística - (Greedy)")
    print(f"Objetivo (suma supervivencia de las amenazas): {total_f:.4f}")
    print(f"Tiempo de ejecución: {elapsed_greedy:.4f} segundos")
    print("Asignación x[i][j]:")
    for i in range(m):
        for j in range(n):
            if x[i][j] > 0:
                print(f"  Buque {i} →amenaza {j}: {x[i][j]} misil(es)")

    print("\np_ij (misil→blanco):")
    for row in p:
        print(" ", ["{:.2f}".format(v) for v in row])

    print("\nV_j (valor de cada blanco):", ["{:.2f}".format(v) for v in V])

# ——— USO ———
if __name__ == "__main__":
    w = [4, 3, 4, 3, 3, 4]
    n = 12
    solve_wta_greedy(w, n, seed=218)

Resultados Heurística - (Greedy)
Objetivo (suma supervivencia de las amenazas): 8.9908
Tiempo de ejecución: 0.0028 segundos
Asignación x[i][j]:
  Buque 0 →amenaza 2: 1 misil(es)
  Buque 0 →amenaza 3: 1 misil(es)
  Buque 0 →amenaza 5: 1 misil(es)
  Buque 0 →amenaza 7: 1 misil(es)
  Buque 1 →amenaza 0: 2 misil(es)
  Buque 1 →amenaza 1: 1 misil(es)
  Buque 2 →amenaza 1: 1 misil(es)
  Buque 2 →amenaza 6: 1 misil(es)
  Buque 2 →amenaza 9: 1 misil(es)
  Buque 2 →amenaza 10: 1 misil(es)
  Buque 3 →amenaza 6: 1 misil(es)
  Buque 3 →amenaza 8: 1 misil(es)
  Buque 3 →amenaza 9: 1 misil(es)
  Buque 4 →amenaza 4: 1 misil(es)
  Buque 4 →amenaza 10: 1 misil(es)
  Buque 4 →amenaza 11: 1 misil(es)
  Buque 5 →amenaza 3: 1 misil(es)
  Buque 5 →amenaza 7: 1 misil(es)
  Buque 5 →amenaza 8: 1 misil(es)
  Buque 5 →amenaza 11: 1 misil(es)

p_ij (misil→blanco):
  ['0.59', '0.53', '0.78', '0.60', '0.56', '0.77', '0.51', '0.72', '0.60', '0.56', '0.60', '0.57']
  ['0.69', '0.75', '0.64', '0.56', '0.57', '0.64', 

### Optimality Gap

El **Optimality Gap** (brecha de optimalidad) es una medida utilizada para evaluar qué tan lejos está una solución heurística (o aproximada) de la solución óptima obtenida por un método exacto. Es especialmente útil cuando se comparan algoritmos como heurísticas (ej. Greedy) frente a modelos exactos resueltos con solvers como CPLEX.

#### Fórmula

La brecha de optimalidad se calcula como:

$
\text{Optimality Gap (\%)} = \frac{Z_{\text{heurístico}} - Z_{\text{óptimo}}}{Z_{\text{óptimo}}} \times 100
$

Donde:

- $ Z_{\text{óptimo}} $ es el valor de la función objetivo obtenido con el método exacto (benchmark).
- $ Z_{\text{heurístico}} $ es el valor de la función objetivo obtenido con el algoritmo heurístico.

#### Interpretación

- **Gap = 0%**: la heurística encontró la solución óptima.
- **Gap > 0%**: la heurística es subóptima; está por encima del óptimo.
- **Gap < 0%**: la heurística supera al modelo exacto (inusual, podría deberse a tolerancia numérica o errores en la formulación del modelo exacto).

In [52]:
import math
import random
import time
import pulp
import pandas as pd

# -----------------------
# GENERADOR DE DATOS
# -----------------------
def generar_datos(m, n, seed=218):
    random.seed(seed)
    p = [[random.uniform(0.5, 0.8) for j in range(n)] for i in range(m)]
    V = [random.uniform(5, 10) for j in range(n)]
    return p, V

# -----------------------
# SOLUCIÓN EXACTA CON PWL
# -----------------------
def solve_wta_pwl(w, p, V, K=70):
    m, n = len(w), len(p[0])
    q = [[1 - p[i][j] for j in range(n)] for i in range(m)]
    y_min = [sum(w[i] * math.log(q[i][j]) for i in range(m)) for j in range(n)]
    breaks = {j: [y_min[j] + k * (0.0 - y_min[j]) / K for k in range(K+1)] for j in range(n)}
    f_vals = {j: [V[j]*math.exp(y) for y in breaks[j]] for j in range(n)}

    model = pulp.LpProblem("WTA_PWL", pulp.LpMinimize)

    x = pulp.LpVariable.dicts("x", (range(m), range(n)), lowBound=0, cat="Integer")
    y = {j: pulp.LpVariable(f"y_{j}", lowBound=y_min[j], upBound=0.0) for j in range(n)}
    f_j = {j: pulp.LpVariable(f"f_{j}", lowBound=min(f_vals[j]), upBound=max(f_vals[j])) for j in range(n)}
    lam = {(j,k): pulp.LpVariable(f"lam_{j}_{k}", lowBound=0) for j in range(n) for k in range(K+1)}

    for j in range(n):
        model += pulp.lpSum(lam[(j,k)] for k in range(K+1)) == 1
        model += y[j] == pulp.lpSum(breaks[j][k] * lam[(j,k)] for k in range(K+1))
        model += f_j[j] == pulp.lpSum(f_vals[j][k] * lam[(j,k)] for k in range(K+1))
        model += y[j] == pulp.lpSum(x[i][j] * math.log(q[i][j]) for i in range(m))

    for i in range(m):
        model += pulp.lpSum(x[i][j] for j in range(n)) <= w[i]

    model += pulp.lpSum(f_j[j] for j in range(n))

    start = time.time()
    model.solve(pulp.CPLEX_CMD(path="/Applications/CPLEX_Studio_Community2211/cplex/bin/arm64_osx/cplex", msg=False))
    elapsed = time.time() - start

    sol = [[int(round(pulp.value(x[i][j]))) for j in range(n)] for i in range(m)]
    obj = pulp.value(model.objective)
    return sol, obj, elapsed

# -----------------------
# SOLUCIÓN GREEDY
# -----------------------
def solve_wta_greedy(w, p, V):
    m, n = len(w), len(p[0])
    q = [[1 - p[i][j] for j in range(n)] for i in range(m)]
    x = [[0 for j in range(n)] for i in range(m)]
    remaining = w[:]
    start = time.time()

    while True:
        best_delta = float("inf")
        best_i, best_j = None, None

        for i in range(m):
            if remaining[i] == 0:
                continue
            for j in range(n):
                current_yj = sum(x[k][j] * math.log(q[k][j]) for k in range(m))
                current_fj = V[j] * math.exp(current_yj)
                proposed_yj = current_yj + math.log(q[i][j])
                proposed_fj = V[j] * math.exp(proposed_yj)
                delta = proposed_fj - current_fj

                if delta < best_delta:
                    best_delta = delta
                    best_i, best_j = i, j

        if best_i is None:
            break
        x[best_i][best_j] += 1
        remaining[best_i] -= 1

    total_f = sum(V[j] * math.exp(sum(x[i][j] * math.log(q[i][j]) for i in range(m))) for j in range(n))
    elapsed = time.time() - start
    return x, total_f, elapsed

# -----------------------
# COMPARACIÓN UNIFICADA
# -----------------------
w = [12, 14, 16, 12]
n = 24
p, V = generar_datos(len(w), n, seed=218)

sol_pwl, obj_pwl, t_pwl = solve_wta_pwl(w, p, V)
sol_greedy, obj_greedy, t_greedy = solve_wta_greedy(w, p, V)

gap = 100 * (obj_greedy - obj_pwl) / obj_pwl

# Resultados organizados
resultados = {
    "Método": ["Exacto (PWL)", "Greedy"],
    "Valor objetivo": [obj_pwl, obj_greedy],
    "Tiempo (seg)": [t_pwl, t_greedy],
    "Optimality Gap (%)": [0, gap]
}
df_resultados = pd.DataFrame(resultados)

df_resultados

TypeError: type NoneType doesn't define __round__ method